In [44]:
import pandas as pd
import glob
import csv
import os

In [45]:
def get_description(maquina):
    df = pd.read_csv(maquina)
    return df.iloc[0]['Descrição']
        
def check_names(nome_lista, maquina_atual):
    return nome_lista in maquina_atual

def filter_name(nome_lista):
    return nome_lista.replace(";", "")

In [50]:
def listagem_maquinas():
    # Lê todos os caminhos para os arquivos das máquinas
    maquinas = glob.glob('maquinas/*.csv')
    
    # Lê a listagem de nomes das máquinas
    nomes = pd.read_csv('nomes/nomes_maquinas.csv')
    
    # Cria arquivo de saida 
    with open('lista.csv', 'w', encoding='UTF-8') as csvfile:
        filewriter = csv.writer(
            csvfile, 
            lineterminator='\n',
            delimiter=',',
            quotechar='|', 
            quoting=csv.QUOTE_MINIMAL
        )
        
        # Cria colunas
        filewriter.writerow(['Nome', 'Descricao'])

        # Percorre o nome de todas as máquinas
        for i in range(nomes.shape[0]):
            
            # Limpa o nome da máquina
            nome_puro = filter_name(nomes.iloc[i][0])

            # Procura a máquina nos arquivos individuais
            for j in range(len(maquinas)):
                
                # Caso exista, adiciona a sua descrição
                if check_names(nome_puro, maquinas[j]) == True:
                    descricao = get_description(maquinas[j])
                    filewriter.writerow([nome_puro,descricao])
                    break
                    
            # Caso não exista, adiciona informação de não exportado
            else:
                descricao = "nao exportado"
                filewriter.writerow([nome_puro,descricao])


In [51]:
def adiciona_novas_colunas():
    df = pd.read_csv('lista.csv')
    
    # remove a coluna de Descrição
    df = df.drop(['Descricao'], axis=1) 
    
    # Adiciona as seguintes colunas
    df['Modelo'] = ''
    df['Setor'] = ''
    df['Sala'] = ''
    
    df.to_csv('nova_lista.csv')

In [83]:
# def adiciona_informacoes(caminho):
caminho = 'infos/Relatório de Microcomputadores agupados por UL (Endereço).csv'
count =0

def is_number(row):
    try:
        int(row[0])
        return True
    except ValueError:
        return False

with open('teste.csv', 'w', encoding='utf-8') as csvfile:
    filewriter = csv.writer(
            csvfile, 
            lineterminator='\n',
            delimiter=',',
            quotechar='|', 
            quoting=csv.QUOTE_MINIMAL
        )
        
    # Cria colunas
    filewriter.writerow(['Patrimonio', 'Patrimonio Antigo', 'Setor'])
    with open(caminho, 'r', encoding='utf-8') as maquinas:
        df = pd.read_csv('nova_lista.csv')
        for row in maquinas:
            index = 0
            if 'U.A.:' in row:
                setor = row[16:-16]
    #             print(setor)
            if 'Endereço:' in row:
                endereco = row[23:-13]
                endereco = endereco.replace(' - SAFS QD2 LT3','')
#                 print(endereco)
            if '3000' in row:
                index = row.find('3000')
#                 print(index)
                patrimonio = row[index:index+8]
                print(patrimonio)

30001480
30002864
30003654
30001462
30001485
30001493
30001644
30001456
30001459
30001483
30001488
30001349
30001359
30001360
30001364
30001365
30003294
30001633
30001639
30001642
30001647
30001653
30001398
30001404
30001409
30001415
30001438
30001610
30003656
30001538
30001270
30001273
30001276
30001553
30001575
30001418
30001562
30001569
30001504
30001557
30001558
30001703
30001706
30001262
30001268
30001453
30001546
30001547
30001373
30003653
30001380
30001385
30001388
30001391
30001690
30001694
30001660
30001665
30003615
30001681
30001682
30001842
30001843
30001845
30001848
30001853
30001856
30001948
30001933
30003588
30001943
30003650
30001599
30003623
30003005
30003613
30001793
30003610
30001795
30003624
30001811
30003648
30003626
30003582
30001160
30001963
30001966
30001972
30001976
30003661
30001958
30002352
30003498
30001833
30002340
30003587
30003602
30003618
30003622
30001737
30001739
30001740
30001745
30001708
30001720
30001752
30001768
30001748
30001775
30003612
30001712
3

In [52]:
listagem_maquinas()

In [53]:
adiciona_novas_colunas()